In [1]:
!pip3 install -r requirements.txt

     |################################| 3.4 MB 18.0 MB/s eta 0:00:01
     |################################| 325 kB 31.9 MB/s eta 0:00:01
     |################################| 3.3 MB 17.3 MB/s eta 0:00:01
     |################################| 603 kB 34.8 MB/s eta 0:00:01
     |################################| 67 kB 7.7 MB/s  eta 0:00:01
     |################################| 9.5 MB 33.5 MB/s eta 0:00:01
     |################################| 86 kB 11.3 MB/s eta 0:00:01
     |################################| 106 kB 35.1 MB/s eta 0:00:01
     |################################| 25.6 MB 22.2 MB/s eta 0:00:01
     |################################| 133 kB 33.5 MB/s eta 0:00:01
     |################################| 211 kB 27.6 MB/s eta 0:00:01
     |################################| 1.1 MB 38.2 MB/s eta 0:00:01
     |################################| 503 kB 40.5 MB/s eta 0:00:01
     |################################| 159 kB 31.7 MB/s eta 0:00:01
     |#############################

In [2]:
from utils import get_geo880, get_satoh_data, get_satoh2_data, get_satoh_data_unseen, get_my_data, get_split, batch_predict

In [55]:
#datasets = get_geo880()
datasets = get_split(4)
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'logical_form'],
        num_rows: 2531
    })
    test: Dataset({
        features: ['text', 'logical_form'],
        num_rows: 133
    })
    val: Dataset({
        features: ['text', 'logical_form'],
        num_rows: 281
    })
})

In [56]:
datasets["train"][0]

{'text': 'The Court holds that the respondent State is to pay the applicant, within three months from the date on which the judgment becomes final in accordance with Article 44 § 2 of the Convention, EUR 9,600 (nine thousand six hundred euros) in respect of non-pecuniary damage and EUR 32 (thirty two euros) in respect of costs and expenses, plus any tax that may be chargeable to be converted into the currency of the respondent State at the rate applicable at the date of settlement',
 'logical_form': 'Court(\\a)^Decide(\\b)^Must(\\c)^DecidedObject(\\b, \\c)^Pay(\\d)^MustObject(\\c, \\d)^State(\\e)^Respondent(\\e)^PaidSubject(\\d, \\e)^Timeframe(\\f)^TimeframeLength(\\f, 3 months)^TimeframeType(\\f, within)^TimeframeEvent(\\f, \\c)^Money(\\h)^PaidObject(\\d, \\h)^Money(\\i)^MemberOf(\\i, \\h)^MoneyAmount(\\i, 9,600 )^Money(\\j)^MemberOf(\\j, \\h)^MoneyAmount(\\j, 32 )^Applicant(\\k)^PaidTo(\\d, \\k))^BecomeFinal(\\l)^BecomeFinalObject(\\l, \\b)^TimeframeStart(\\f, \\l)^Convention(\\m)^Co

In [57]:
num_texts = len(datasets["train"]["text"])
num_logic = len(datasets["train"]["logical_form"])
print("Num texts: {} Num logical forms {}".format(num_texts, num_logic))
text_avg = 0.0
logic_avg = 0.0
for item in datasets["train"]:
    text_avg += len(item["text"])
    logic_avg += len(item["logical_form"])
    
text_avg /= float(num_texts)
logic_avg /= float(num_logic)

print("Avg text length: {} Avg logical form length {}".format(text_avg, logic_avg))

Num texts: 2531 Num logical forms 2531
Avg text length: 322.1027261951798 Avg logical form length 737.4413275385223


In [58]:
MAX_SOURCE_LENGTH = 1024
MAX_TARGET_LENGTH = 1024
USES_PADDING = "max_length" #False
DO_IGNORE_PAD_TOKEN_FOR_LOSS = True

In [59]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM
model_name = "facebook/bart-base"#"t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_config(config)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "

In [60]:
def preprocess_function(data):
        input_encodings = tokenizer.batch_encode_plus(data['text'], padding=USES_PADDING, truncation=True)
        target_encodings = tokenizer.batch_encode_plus(data['logical_form'], padding=USES_PADDING, truncation=True)

        labels = target_encodings['input_ids']
        if USES_PADDING == "max_length" and DO_IGNORE_PAD_TOKEN_FOR_LOSS:
            labels = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels
            ]

        encodings = {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': labels,
        }
        return encodings

In [61]:
datasets = datasets.map(preprocess_function, batched=True)
columns = ['input_ids', 'labels','attention_mask',] 
datasets.set_format(type='torch', columns=columns)

In [62]:
num_texts = len(datasets["train"]["input_ids"])
num_logic = len(datasets["train"]["labels"])
print("Num texts: {} Num logical forms {}".format(num_texts, num_logic))
text_avg = 0.0
logic_avg = 0.0
for item in datasets["train"]:
    text_avg += len(item["input_ids"])
    logic_avg += len(item["labels"])
    
text_avg /= float(num_texts)
logic_avg /= float(num_logic)

print("Avg text length: {} Avg logical form length {}".format(text_avg, logic_avg))

Num texts: 2531 Num logical forms 2531
Avg text length: 1024.0 Avg logical form length 1024.0


In [63]:
from transformers import Trainer, TrainingArguments

model_name = "my_model_v1.0-split4"

training_args = TrainingArguments(
    output_dir='./models/' + model_name,          
    num_train_epochs=10,           
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,   
    warmup_steps=100,               
    weight_decay=0.01,              
    logging_dir='./logs',  
    save_steps=20000,
)

trainer = Trainer(
    model=model,                       
    args=training_args,                  
    train_dataset=datasets['train'],        
    eval_dataset=datasets['val']   
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [64]:
import os
if not os.path.exists("./trained_models/{}".format(model_name)):
    if os.path.exists("./models/{}/checkpoint-20000".format(model_name)):
        trainer.train("./models/{}/checkpoint-20000".format(model_name))
    else:
        trainer.train()
else:
    our_model = model.from_pretrained("./trained_models/{}".format(model_name))
    trainer = Trainer(
    model=our_model,                       
    args=training_args,                  
    train_dataset=datasets['train'],        
    eval_dataset=datasets['val']   
    )

loading configuration file ./trained_models/my_model_v1.0-split4/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id

In [26]:
if not os.path.exists("./trained_models/{}".format(model_name)):
    trainer.save_model("./trained_models/{}".format(model_name))

Saving model checkpoint to ./trained_models/my_model_v1.0-split4
Configuration saved in ./trained_models/my_model_v1.0-split4/config.json
Model weights saved in ./trained_models/my_model_v1.0-split4/pytorch_model.bin


In [65]:
original_texts, original_logical_forms, predicted_logical_forms = batch_predict(datasets["test"], trainer, tokenizer, limit=50)

The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: logical_form, text.
***** Running Prediction *****
  Num examples = 1
  Batch size = 1


The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: logical_form, text.
***** Running Prediction *****
  Num examples = 1
  Batch size = 1
The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: logical_form, text.
***** Running Prediction *****
  Num examples = 1
  Batch size = 1
The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: logical_form, text.
***** Running Prediction *****
  Num examples = 1
  Batch size = 1
The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: logical_form, text.
***** Running Prediction *****
  Num examples = 1
  Batch size = 1
The following columns in the test set  don't have a corresponding argument in `BartForCondit

In [ ]:
original_texts2, original_logical_forms2, predicted_logical_forms2 = batch_predict(split["test"], trainer, tokenizer, limit=37)

In [24]:
len(predicted_logical_forms)

49

In [66]:
count = 0
for i, form in enumerate(original_logical_forms):
    if i == 49:
        break
    if predicted_logical_forms[i].replace("Ġ", " ") != form:
        print(original_texts[i])
        print("\n")
        print(form)
        print("\n")
        print(predicted_logical_forms[i].replace("Ġ", " "))
        print("\n\n\n")
    else:
        count += 1
        
print(count)

The Court dismisses the remainder of the applicant’s claims for just satisfaction


Court(\a)^Decide(\b)^Dismiss(\c)^DecidedObject(\b, \c)^Claims(\d)^Rest(\e)^RestObject(\e, \d)^DismissedObject(\c, \e)^Reason(\b, just satisfaction)^Applicant(\f)^Owner(\d, \f)


Court(\a)^Decide(\b)^Must(\(\Objectd,^BelidedObject(\d, \f)^Article(\)d)^ConObjectc)^ConObject(\e) \f,^Ex(\.(\To(\d, \f)^Con.i) \))^Timeants(\d)^Con(\i, \f)^




The Court dismisses unanimously the remainder of the applicants’ claims for just satisfaction.


Court(\a)^Decide(\b)^Dismiss(\c)^DecidedObject(\b, \c)^Claims(\d)^Rest(\e)^RestObject(\e, \d)^DismissedObject(\c, \e)^Reason(\b, just satisfaction)^Applicants(\f)^Owner(\d, \f)^VotesFor(\b, 7)^VotesAgainst(\b, 0)


Court(\a)^Decide(\b)^Must(\(\Objectd,^ExidedObject(\d, \f)^Article(\)d)^ConObjectc)^ConObject(\e) \f,^Ex(\.(\Object(\d, \e)^Con.i) \))^Timeants(\d)^Con(\i, \h)^Conotes within(\i) \)




The Court holds that simple interest at an annual rate of 7% shall be payable 

In [17]:
[pred.replace("Ġ", " ") for pred in predicted_logical_forms]

['is_a_parent_of(X1,X2) :- is_that(X1,X3),is_of_type(X3,X2).',
 'not(notifies(X2,X1)) :- is_an_eligible_asset(X1),it_is_the_end_of(X2)',
 'requests(X1,X2) :- terminates_at(X1,X4,X3),of_Default_occurs_with_respect_to_at(X5,X1,X3),not(is_on_or_before(X1,X5),happens_at(X2,X4))',
 'pays_to(X1,X2,X3) :- is_on_or_before(X4,X5),is_not_more_than_days_after(X4,X1,X5),is_undertaken_as_part_of(X2,X5),is_defaulted_on(X5,X3)',
 'is_an_active_asset(X1) :- is_remedied(X2),has_paid_itself(X2,X1).',
 'the_Schedule_specifies_that(X1) :- occurs_at(X2,X3),not(is_of_type(X2,X1))',
 'happens_at(X1,X2) :- is_terminated(X1),it_is_permitted_that(X1),forall(happens_at(X3,X1), occurs_at(X3,X2))',
 'is_on_or_before(X1,X2) :- is_a_small_business_entity_as_informed_by(X3,X4),is_interrupted_between_and(X3,X1,X4),occurs_at(X4,X2),not(of_Default_occurs_with_respect_to_at(X1,X4,X2))',
 'is_defaulted_on(X1,X2) :- of_Default_occurs_with_respect_to_at(X1,X4,X3),of_Default_occurs_with_respect_to_at(X5,X1,X3),not(is_on_or_b

In [ ]:
from utils import EXCLUDE_TYPES
for i, point in enumerate(original_logical_forms):
        name = point[:point.find("(")]
        if name in EXCLUDE_TYPES:
            print(point)
            #print(predicted_logical_forms[i].replace("Ġ", " "))

is_a_party_of('the restaurant','an event')
designates_at_that('a party','last night','emergency assist systems can be manually disabled')
has_paid_itself('an agent','withholding taxes')
is_a_member_of('a senator','the membership organization')
designates_at_that('the owner of the assets','last night','emergency assist systems can be manually disabled')
designates_at_that('the taxpayer',meetings,'an eventuality')
designates_at_that('the restaurant',meetings,'emergency assist systems can be manually disabled')
designates_at_that('a party',meetings,'the driver can engage a manual speed limitation function')
it_is_the_end_of('a date')
designates_at_that('the restaurant','a time','an eventuality')
is_a_member_of('an individual','the membership organization')
has_paid_itself('the sole agent','a fee')
is_a_small_business_entity_as_informed_by('the taxpayer','the secretariat')
designates_at_that('the taxpayer','a time','the driver can engage a manual speed limitation function')
designates_at_t